# Modifying phrases

* Annotate modifying phrases by using parse tree instead of humans or GPT

  - "prepositional phrases"
    ```
    PP
    ```
    
  - "single word modifiers",
    ```
    ADJ: "adjective",
    ADV: "adverb",
  
    RB: "adverb",
    RBR: "adverb, comparative",
    RBS: "adverb, superlative",
    RP: "adverb, particle",
    JJ: "adjective (English), other noun-modifier (Chinese)",
    JJR: "adjective, comparative",
    JJS: "adjective, superlative",
    ```

  
  - "multiplying and embedding modifiers"
  

In [7]:
import pandas as pd
import utils

In [3]:
df = pd.read_csv("data/human_gpt_verified/Modifying_phrases.csv")

In [5]:
len(df)

87

In [6]:
df.columns

Index(['sentence_id', 'technique', 'text', 'feature_id', 'props_a20',
       'props_a21', 'props_a22', 'annotator_consistency',
       'props_gpt4_majority', 'res_1.0_1', 'gpt_props_1.0_1', 'res_1.0_2',
       'gpt_props_1.0_2', 'res_1.0_3', 'gpt_props_1.0_3',
       'gpt3.5_1.0_consistency', 'res_0.2_1', 'gpt_props_0.2_1', 'res_0.2_2',
       'gpt_props_0.2_2', 'res_0.2_3', 'gpt_props_0.2_3',
       'gpt3.5_0.2_consistency', 'gpt3.5_0.2_majority', 'humans isCorrect',
       'gpt isCorrect', 'comments', 'ground truth'],
      dtype='object')

In [8]:
connection = utils.connectDB()

In [18]:
ids = tuple(df['sentence_id'])

In [25]:
query = 'SELECT id,text, parseTree FROM ebdb.sample_sentences WHERE id in '+str(ids)+';'
parseTrees = pd.read_sql(query, connection)

/var/folders/vt/g1t9ds3s01z05zs4qg2dngk80000gn/T/ipykernel_58175/2940145906.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  parseTrees = pd.read_sql(query, connection)


In [26]:
parseTrees

,id,text,parseTree
0,253,Either that or they can go whine to the New Yo...,(S (CC Either) (NP (DT that)) (CC or) (NP (PRP...
1,263,While Obama pardoned drug dealers and locked u...,(S (SBAR (IN While) (S (NP (NNP Obama)) (VP (V...
2,282,"And if Trumpism is not the future of the GOP, ...",(S (CC And) (SBAR (IN if) (S (NP (NNP Trumpism...
3,307,"Either you stand with BDS, Hamas, blood libels...",(S (CC Either) (NP (PRP you)) (VP (VBP stand) ...
4,311,"The Vatican presents a choice: the world, or C...",(S (NP (DT The) (NNP Vatican)) (VP (VBZ presen...
...,...,...,...
82,12626,It did what it is designed to do.,(S (NP (PRP It)) (VP (VBD did) (SBAR (WHNP (WP...
83,12751,"To keep up everyone’s morale, the museum staff...",(S (S (VP (TO To) (VP (VB keep) (PRT (RP up)) ...
84,13007,"In a series of tweets on Thursday, President T...",(S (PP (IN In) (NP (NP (DT a) (NN series)) (PP...
85,14506,"When asked about the threats, Haig said, “I’ve...",(S (SBAR (WHADVP (WRB When)) (S (VP (VBN asked...


In [39]:
single_words = ["(RB ",
"(RBR ",
"(RBS ",
"(RP ",
"(JJ ",
"(JJR ",
"(JJS ",]
prepositional = "(PP "

In [47]:
def singleWords(s):
    count = 0
    for sw in single_words:
        if sw in s:
            count+=1
    return count
def prepos(s):
    if prepositional in s:
        return 1
    else:
        return 0

In [48]:
parseTrees['single_word'] = parseTrees['parseTree'].apply(singleWords)
parseTrees['prepositional'] = parseTrees['parseTree'].apply(prepos)

In [50]:
parseTrees['multiplying'] = parseTrees.apply(lambda r: r['single_word']+r['prepositional'] > 2, axis=1)

In [52]:
parseTrees['multiplying'] = parseTrees['multiplying'].apply(lambda x: int(x))

In [53]:
parseTrees

,id,text,parseTree,single_word,prepositional,multiplying
0,253,Either that or they can go whine to the New Yo...,(S (CC Either) (NP (DT that)) (CC or) (NP (PRP...,3,1,1
1,263,While Obama pardoned drug dealers and locked u...,(S (SBAR (IN While) (S (NP (NNP Obama)) (VP (V...,1,0,0
2,282,"And if Trumpism is not the future of the GOP, ...",(S (CC And) (SBAR (IN if) (S (NP (NNP Trumpism...,2,1,1
3,307,"Either you stand with BDS, Hamas, blood libels...",(S (CC Either) (NP (PRP you)) (VP (VBP stand) ...,0,1,0
4,311,"The Vatican presents a choice: the world, or C...",(S (NP (DT The) (NNP Vatican)) (VP (VBZ presen...,0,0,0
...,...,...,...,...,...,...
82,12626,It did what it is designed to do.,(S (NP (PRP It)) (VP (VBD did) (SBAR (WHNP (WP...,0,0,0
83,12751,"To keep up everyone’s morale, the museum staff...",(S (S (VP (TO To) (VP (VB keep) (PRT (RP up)) ...,2,0,0
84,13007,"In a series of tweets on Thursday, President T...",(S (PP (IN In) (NP (NP (DT a) (NN series)) (PP...,1,1,0
85,14506,"When asked about the threats, Haig said, “I’ve...",(S (SBAR (WHADVP (WRB When)) (S (VP (VBN asked...,1,1,0
